# CÓDIGO
* Se realizan los siguientes análisis:
    * 1.Detección de valores no numéricos 
    * 2.Remoción de elementos por digestión
    * 3.Reemplaza data LD a la mitad
    * 4.Convertir PPB o PCT a PPM
    * 5.Limpiado de data de nuevo
    * 6.Añadir Ln
    * 7.Ordenamiento de columnas
    * 8.Remoción de outlier 1 y 2
    * 9.Separar CSV
    * 10.Prueba de normalidad y zscore
    * 11.Conversión de valores Ln a base (Paramétrico y no paramétricos)
    * 12.Parámetros univariados (Paramétrico y no paramétrico)
    * 13.Exportación de shapefiles
* Parámetros a escribir:
    * La ruta del archivo de entrada (no se recomienda modificar el nombre)
    * Número de columna para iniciar el análisis (se recomienda la anterior a los elementos)
    * Mantener la columna de dominios como la última (para ello el paso 7)
    * Número de decimales
    * Modificar la cantidad de figuras por columna y fila en caso de los gráficos

In [ ]:
import pandas as pd

# Especifica la ruta del archivo Excel (reemplaza 'tu_archivo.xlsx' con el nombre de tu archivo)
archivo_excel = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/UNIDO.xlsx"

# Lee el archivo Excel en un DataFrame de pandas
df = pd.read_excel(archivo_excel)

# Especifica la ruta de salida para el archivo CSV (puedes cambiar el nombre si lo deseas)
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/UNIDO.csv"

# Guarda el DataFrame en un archivo CSV
df.to_csv(archivo_csv, index=False)

print(f"Archivo CSV guardado exitosamente en: {archivo_csv}")

# 1. LEER VALORES NO NUMERICOS
* El codigo va a leer si es >,<, nulo o 0
* Sumará sus valores para luego dividirlo entre el total de celdas por columna
* Eliminará los valores que no cumplan el valor limite
* Exportará los valores correctos

In [ ]:
'C:/10mo/1_TESIS 2/1_GEOQUIMICA/1_DOMINIOS/EXCEL/DOMINIO2.csv'

In [ ]:
import pandas as pd

# 1. Importa un dataframe de un archivo csv (reemplaza 'tu_archivo.csv' con tu archivo)
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/UNIDO.csv"
df = pd.read_csv(archivo_csv)
decimal=int(input("Ingrese número de decimales: "))
# 2. Pide el número de la columna desde la cual se iniciará el análisis
numero_columna = int(input(f"Ingrese el número de la columna desde la cual desea iniciar el análisis (0 a {len(df.columns) - 1}): "))

# 3. Pide un valor límite
valor_limite = float(input("Ingrese el valor límite: "))

# Verifica si el número de columna es válido
if 0 <= numero_columna < len(df.columns):
    # 4. A los valores que sean 0 se les pone vacío
    df.iloc[:, numero_columna:] = df.iloc[:, numero_columna:].replace(0, pd.NA)

    # Inicializa una lista para almacenar los resultados que exceden el valor límite
    resultados_exceden_limite = []

    # 5. Para cada columna, cuenta los valores que inician con ">", "<", que empiecen por alguna letra o vacío
    for columna in df.columns[numero_columna:]:
        # Filtra los valores que cumplen con las condiciones especificadas
        valores_mayor = df[columna].astype(str).str.startswith('>')
        valores_menor = df[columna].astype(str).str.startswith('<')
        valores_letra = df[columna].astype(str).str[0].str.isalpha()
        valores_vacios = df[columna].astype(str).str.strip() == ''

        # 6. Muestra los valores por cada columna
        print(f"\nAnálisis de la columna: {columna}")
        print(f"Valores que inician con '>': {valores_mayor.sum()}")
        print(f"Valores que inician con '<': {valores_menor.sum()}")
        print(f"Valores que empiezan con letra: {valores_letra.sum()}")
        print(f"Valores vacíos: {valores_vacios.sum()}")

        # 7. Sumar esos valores y dividirlos por el total por cada columna
        total_celdas = len(df[columna])
        suma_valores = valores_mayor.sum() + valores_menor.sum() + valores_letra.sum() + valores_vacios.sum()
        promedio = suma_valores / total_celdas

        # 8. Comparar el resultado con el valor límite y mostrar los valores límites por columna que excedan el valor límite
        print(f"Promedio: {promedio:.2f}")
        if promedio > valor_limite:
            resultados_exceden_limite.append((columna, promedio))

    # Mostrar los resultados que exceden el valor límite en una sección aparte
    if resultados_exceden_limite:
        print("\nResultados que exceden el valor límite:")
        for resultado in resultados_exceden_limite:
            print(f"Columna: {resultado[0]}, Promedio: {resultado[1]:.2f}")

        # 9. Dar la opción para que escriba qué columnas no quiere eliminar del grupo de columnas que excedieron el límite
        columnas_a_mantener = input("Escriba los nombres de las columnas que no desea eliminar (separadas por coma): ")
        columnas_a_mantener = [col.strip() for col in columnas_a_mantener.split(',')]

        # 10. Eliminar las columnas restantes del grupo
        columnas_a_eliminar = [col[0] for col in resultados_exceden_limite if col[0] not in columnas_a_mantener]
        df = df.drop(columnas_a_eliminar, axis=1)

        # 11. Exportar el dataframe con nombre "DOM_LD"
        df.round(decimal).to_csv('C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/1_DOM_LD.csv', index=False)
        print("El dataframe ha sido exportado con el nombre '1_DOM_LD.csv'.")

else:
    print("Número de columna no válido. Por favor, ingrese un número dentro del rango.")


# 2. Metodos de digestion
* Filtra los elementos en función del tipo de digestión que se ha empleado
* Permite agregar elementos extra

In [ ]:
import pandas as pd

# 1. Importa un dataframe de un archivo csv (reemplaza 'tu_archivo.csv' con tu archivo)
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/1_DOM_LD.csv'
df = pd.read_csv(archivo_csv)

# 2. Pide el número de la columna desde la cual se iniciará el análisis
numero_columna = int(input(f"Ingrese el número de la columna desde la cual desea iniciar el análisis (0 a {len(df.columns) - 1}): "))

# 3. Crea listas llamadas AR, 4A, FAAS, SPF y LBF con nombres específicos
AR = ['Li', 'Sc', 'Mn', 'Re', 'Cu', 'Au', 'Zn', 'Cd', 'Hg', 'Ga', 'In', 'Tl', 'Pb', 'Bi', 'Sb', 'As', 'Se', 'Te']
_4A = ['Na', 'K', 'Rb', 'Cs', 'Be', 'Mg', 'Ca', 'Sr', 'Sc', 'Hf', 'La', 'Ce', 'Mo', 'Pr', 'Mn', 'Re', 'Nd', 'Fe', 'Co', 'Sm', 'Ni', 'Eu', 'Gd', 'Ag', 'Cu', 'Zn', 'Cd', 'Tb', 'Al', 'Ga', 'In', 'Tl', 'Dy', 'Ho', 'Pb', 'P', 'As', 'Sb', 'Bi', 'Er', 'Tm', 'Te', 'Se', 'S', 'Yb', 'Lu']
FAAS = ['Ru', 'Os', 'Rh', 'Ir', 'Pd', 'Pt', 'Ag', 'Au']
SPF = ['Li', 'Be', 'Ti', 'Zr', 'V', 'Nb', 'Ta', 'Cr', 'W', 'Co', 'Ni', 'B', 'Si', 'Sn']
LBF = ['Na', 'K', 'Rb', 'Cs', 'Mg', 'Ca', 'Sr', 'Ba', 'Y', 'Ti', 'Zr', 'Hf', 'La', 'V', 'Nb', 'Ta', 'Ce', 'Pr', 'W', 'Cr', 'Nd', 'Fe', 'Sm', 'Eu', 'Gd', 'Tb', 'Al', 'Dy', 'Ho', 'Sn', 'Si', 'Er', 'P', 'Tm', 'S', 'Yb', 'Lu']

# 8. Pida una entrada de texto llamada "Ensayo"
ensayo = input("Ingrese el tipo de ensayo (AR, _4A, FAAS, SPF, LBF): ")

# 9. Muestra el nombre de cada columna desde la que se iniciará el análisis
print(f"Nombres de las columnas desde la columna {numero_columna}:")
for i, columna in enumerate(df.columns[numero_columna:]):
    print(f"{i}. {columna}")

# 10. Coincidir la entrada "Ensayo" con una de las 5 listas creadas al inicio
if ensayo == "AR":
    lista_ensayo = AR
elif ensayo == "_4A":
    lista_ensayo = _4A
elif ensayo == "FAAS":
    lista_ensayo = FAAS
elif ensayo == "SPF":
    lista_ensayo = SPF
elif ensayo == "LBF":
    lista_ensayo = LBF
else:
    print("Tipo de ensayo no reconocido.")

# 11. Compare los elementos de la lista que cumplía el criterio con las dos primeras letras de los nombres de la columna. Muestre aquellos que no coincidan, sepáralos en una lista aparte
columnas_no_coincidentes = [col for col in df.columns[numero_columna:] if col[:2].lower() not in [elem[:2].lower() for elem in lista_ensayo]]

print("Columnas cuyas dos primeras letras no coinciden con la lista seleccionada:")
for i, col in enumerate(columnas_no_coincidentes):
    print(f"{i}. {col}")

# 12. Dar la opción para que escriba qué columnas no quiere eliminar del grupo de columnas que no eran coincidentes.
columnas_a_mantener_str = input("Escriba el número de las columnas que no quiere eliminar, separadas por comas (si no desea mantener ninguna, presione Enter): ")
if columnas_a_mantener_str:
    columnas_a_mantener = [int(col) for col in columnas_a_mantener_str.split(',')]
else:
    columnas_a_mantener = []

# 13. Eliminar las columnas restantes del grupo
columnas_a_eliminar = [col for i, col in enumerate(columnas_no_coincidentes) if i not in columnas_a_mantener]
df = df.drop(columns=columnas_a_eliminar, axis=1)

# 14. Exportar el dataframe con nombre "DOM_DIG"
df.round(decimal).to_csv('C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/2_DOM_DIG.csv', index=False)

print("Proceso completado. Se ha exportado el dataframe con nombre '2_DOM_DIG.csv'.")


# 3. LIMPIAR LD
* Reemplaza los valores vacíos o 0 a nulos
* Reduce a la mitad los valores con > o <

In [ ]:
import pandas as pd

# 1. Importar un DataFrame desde un archivo CSV
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/2_DOM_DIG.csv'  # Cambia por el nombre de tu archivo CSV
df1 = pd.read_csv(archivo_csv)
df = df1.copy()  # Se crea una copia del DataFrame original para no modificarlo directamente

decimal = int(input("Ingrese número de decimales: "))

# 2. Pide el número de la columna desde la cual se iniciará el análisis
columna_inicial = int(input("Ingrese el número de la columna desde la cual se iniciará el análisis: ")) - 1

# 3. Por cada columna, reemplazar los valores 0 o vacíos a nulo (excluyendo la última columna)
def reemplazar_nulos(valor):
    return pd.NA if valor == 0 or pd.isna(valor) or (isinstance(valor, str) and (valor.strip() == '' or valor[0].isalpha())) else valor

# Aplicar la función a cada elemento del DataFrame excluyendo la última columna
df.iloc[:, columna_inicial:-1] = df.iloc[:, columna_inicial:-1].applymap(reemplazar_nulos)

# 4. Si hay celdas que empiecen con símbolo ">" o "<", eliminar el símbolo y convertirlo a número
#    Luego, reemplazar ese número por su mitad
def procesar_celda(valor):
    if pd.notna(valor) and isinstance(valor, str) and (valor.startswith('>') or valor.startswith('<')):
        numero = float(valor[1:])
        return round(numero / 2, decimal)
    return valor

for columna in df.columns[columna_inicial+1:-1]:
    df[columna] = df[columna].map(procesar_celda)

df.round(decimal).to_csv('C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/3_DOM_NOLD.csv', index=False)

print("Proceso completado. Se ha exportado el dataframe con nombre '3_DOM_NOLD.csv'.")


# 4. CONVERTIR PPB O PCT A PPM

In [ ]:
'C:/10mo/1_TESIS 2/1_GEOQUIMICA/1_DOMINIOS/EXCEL/DOM_LD.csv'

In [ ]:
import pandas as pd

# 1. Importar un dataframe desde un archivo CSV
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/3_DOM_NOLD.csv'  # Reemplaza con la ruta y nombre de tu archivo CSV
dataframe = pd.read_csv(archivo_csv)
decimal=int(input("Ingrese número de decimales: "))
# 2. Pedir el número de la columna desde la cual se iniciará el análisis
inicio_analisis = int(input("Ingrese el número de la columna desde la cual se iniciará el análisis: "))

# 3. y 4. Procesar las columnas que terminan en "ppb" y "ppt"
for i in range(inicio_analisis, len(dataframe.columns)):
    nombre_columna = dataframe.columns[i]
    
    if nombre_columna.endswith("ppb"):
        # Cambiar el nombre de la columna y dividir por 1000
        nuevo_nombre = nombre_columna[:-3] + "ppm"
        dataframe.rename(columns={nombre_columna: nuevo_nombre}, inplace=True)
        dataframe[nuevo_nombre] = round(dataframe[nuevo_nombre] / 1000, decimal)
    
    elif nombre_columna.endswith("pct"):
        # Cambiar el nombre de la columna y dividir por 1000000
        nuevo_nombre = nombre_columna[:-3] + "ppm"
        dataframe.rename(columns={nombre_columna: nuevo_nombre}, inplace=True)
        dataframe[nuevo_nombre] = round(dataframe[nuevo_nombre] * 10000, decimal)

print(dataframe)
# 5. Exportar el dataframe en CSV sin incluir el índice
dataframe.round(decimal).to_csv("C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/4_DOM_PP.csv", index=False)


# 5. VOLVER A LIMPIAR LA DATA
* Se hace en caso los ppb o pct den valores 0.000

In [ ]:
import pandas as pd

# 1. Importa un dataframe de un archivo csv (reemplaza 'tu_archivo.csv' con tu archivo)
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/4_DOM_PP.csv"
df = pd.read_csv(archivo_csv)
decimal=int(input("Ingrese número de decimales: "))
# 2. Pide el número de la columna desde la cual se iniciará el análisis
numero_columna = int(input(f"Ingrese el número de la columna desde la cual desea iniciar el análisis (0 a {len(df.columns) - 1}): "))

# 3. Pide un valor límite
valor_limite = float(input("Ingrese el valor límite: "))

# Verifica si el número de columna es válido
if 0 <= numero_columna < len(df.columns):
    # 4. A los valores que sean 0 se les pone vacío
    df.iloc[:, numero_columna:] = df.iloc[:, numero_columna:].replace(0, pd.NA)

    # Inicializa una lista para almacenar los resultados que exceden el valor límite
    resultados_exceden_limite = []

    # 5. Para cada columna, cuenta los valores que inician con ">", "<", que empiecen por alguna letra o vacío
    for columna in df.columns[numero_columna:]:
        # Filtra los valores que cumplen con las condiciones especificadas
        valores_mayor = df[columna].astype(str).str.startswith('>')
        valores_menor = df[columna].astype(str).str.startswith('<')
        valores_letra = df[columna].astype(str).str[0].str.isalpha()
        valores_vacios = df[columna].astype(str).str.strip() == ''

        # 6. Muestra los valores por cada columna
        print(f"\nAnálisis de la columna: {columna}")
        print(f"Valores que inician con '>': {valores_mayor.sum()}")
        print(f"Valores que inician con '<': {valores_menor.sum()}")
        print(f"Valores que empiezan con letra: {valores_letra.sum()}")
        print(f"Valores vacíos: {valores_vacios.sum()}")

        # 7. Sumar esos valores y dividirlos por el total por cada columna
        total_celdas = len(df[columna])
        suma_valores = valores_mayor.sum() + valores_menor.sum() + valores_letra.sum() + valores_vacios.sum()
        promedio = round(suma_valores / total_celdas,decimal)

        # 8. Comparar el resultado con el valor límite y mostrar los valores límites por columna que excedan el valor límite
        print(f"Promedio: {promedio:.2f}")
        if promedio > valor_limite:
            resultados_exceden_limite.append((columna, promedio))

    # Mostrar los resultados que exceden el valor límite en una sección aparte
    if resultados_exceden_limite:
        print("\nResultados que exceden el valor límite:")
        for resultado in resultados_exceden_limite:
            print(f"Columna: {resultado[0]}, Promedio: {resultado[1]:.2f}")

        # 9. Dar la opción para que escriba qué columnas no quiere eliminar del grupo de columnas que excedieron el límite
        columnas_a_mantener = input("Escriba los nombres de las columnas que no desea eliminar (separadas por coma): ")
        columnas_a_mantener = [col.strip() for col in columnas_a_mantener.split(',')]

        # 10. Eliminar las columnas restantes del grupo
        columnas_a_eliminar = [col[0] for col in resultados_exceden_limite if col[0] not in columnas_a_mantener]
        df = df.drop(columnas_a_eliminar, axis=1)

        # 11. Exportar el dataframe con nombre "DOM_LD"
        df.round(decimal).to_csv('C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/5_DOM_LIMPIO.csv', index=False)
        print("El dataframe ha sido exportado con el nombre '5_DOM_LIMPIO.csv'.")

else:
    print("Número de columna no válido. Por favor, ingrese un número dentro del rango.")


# 6. AÑADIR COLUMNAS LN

In [ ]:
import pandas as pd
import numpy as np

# Paso 1: Importa un dataframe en csv
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/5_DOM_LIMPIO.csv'  # Reemplaza 'tu_archivo.csv' con el nombre de tu archivo CSV
df = pd.read_csv(archivo_csv)
decimal=int(input("Ingrese número de decimales: "))
# Paso 2: Pide el número de columna desde la cual se hará el análisis hasta la penúltima columna
try:
    inicio_columna = int(input("Ingrese el número de la columna desde la cual se realizará el análisis: "))
except ValueError:
    print("Por favor, ingrese un número válido.")

# Paso 3: Crea a la derecha de cada columna original una nueva columna con logaritmos neperianos
for i in range(inicio_columna, len(df.columns) - 1):
    columna_original = df.columns[i]
    print(columna_original)
    nueva_columna_nombre = "Ln" + columna_original
    df[nueva_columna_nombre] = np.log(df[columna_original]).round(decimal)

# Paso 4: Exporta el dataframe con el nombre "6_DOM_LN"
nombre_archivo_salida = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/6_DOM_LN.csv'
df.round(decimal).to_csv(nombre_archivo_salida, index=False)

print(f"El dataframe con logaritmos neperianos ha sido exportado como '{nombre_archivo_salida}'.")


# 7. ORDENAR COLUMNAS
* Permite seleccionar las columnas para ponerlas al final
* Yo la usé para poner la columna DOM al final (la mayoría del código funciona con la columna de dominio al final)

In [ ]:
import pandas as pd

# Paso 1: Importa un dataframe en csv
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/6_DOM_LN.csv'  # Reemplaza 'tu_archivo.csv' con el nombre de tu archivo CSV
df = pd.read_csv(archivo_csv)
decimal=int(input("Ingrese número de decimales: "))
# Muestra los números y nombres de las columnas actuales
print("Número de columna - Nombre de columna:")
for i, columna in enumerate(df.columns):
    print(f"{i}: {columna}")

# Paso 2: Seleccionar qué columnas cambiarán de posición por número de columna
columnas_a_mover_numeros = input("Ingrese los números de columna que desea mover (separados por coma): ").split(',')

# Validar que los números de columna ingresados sean válidos
try:
    columnas_a_mover_numeros = [int(num.strip()) for num in columnas_a_mover_numeros]
except ValueError:
    print("Por favor, ingrese números de columna válidos. Saliendo.")
    exit()

columnas_a_mover_validas = [col for col in df.columns if df.columns.get_loc(col) in columnas_a_mover_numeros]

if not columnas_a_mover_validas:
    print("No se ingresaron números de columna válidos. Saliendo.")
    exit()

# Mostrar los números y nombres de las columnas que se pueden cambiar de posición
print("Número de columna - Nombre de columna:")
for i, columna in enumerate(columnas_a_mover_validas):
    print(f"{i}: {columna}")

# Copiar las columnas seleccionadas aparte
df_copia = df[columnas_a_mover_validas].copy()

# Eliminar las columnas seleccionadas del dataframe original
df = df.drop(columns=columnas_a_mover_validas)

# Añadir las columnas copiadas al final del dataframe
df = pd.concat([df, df_copia], axis=1)

# Mostrar el dataframe resultante
print("\nDataframe con columnas cambiadas de posición:")
print(df)

# Opcional: Exportar el dataframe resultante a un nuevo archivo CSV
nombre_archivo_salida = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/7_DOM_PRESEP.csv'
df.round(decimal).to_csv(nombre_archivo_salida, index=False)
print(f"\nEl dataframe modificado ha sido exportado como '{nombre_archivo_salida}'.")


# CREAR HISTOGRAMAS CON DATA OUTLIER

In [ ]:
'C:/10mo/1_TESIS 2/1_GEOQUIMICA/1_DOMINIOS/EXCEL/7_DOM_PRESEP.csv'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Paso 1: Importar un dataframe en csv
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/8_DOM_OUT5.csv'  # Reemplaza 'tu_archivo.csv' con el nombre de tu archivo CSV
df = pd.read_csv(archivo_csv)

# Paso 2: Seleccionar desde qué columna se iniciará el análisis
try:
    inicio_columna = int(input("Ingrese el número de la columna desde la cual se iniciará el análisis: "))
except ValueError:
    print("Por favor, ingrese un número válido.")

# Validar que el número de columna ingresado sea válido
if inicio_columna < 0 or inicio_columna >= len(df.columns):
    print("Número de columna no válido. Saliendo.")
    exit()

# Filtrar las columnas a partir del inicio_columna
columnas_analisis = df.columns[inicio_columna:]

# Paso 3: Seleccionar una columna para hacer histogramas
columna_histograma = input("Ingrese el nombre de la columna para hacer histogramas: ")

# Validar que la columna seleccionada exista en el dataframe
if columna_histograma not in df.columns:
    print("La columna seleccionada no existe en el dataframe. Saliendo.")
    exit()

# Obtener valores únicos de la columna de histograma
valores_unicos = df[columna_histograma].unique()

# Paso 4: Graficar histogramas solo para las columnas no seleccionadas
for valor_unico in valores_unicos:
    # Filtrar el dataframe por el valor único en la columna de histograma
    df_filtrado = df[df[columna_histograma] == valor_unico]

    # Crear subplots para cada valor único
    fig, axs = plt.subplots(4, 5, figsize=(25, 25))

    # Contadores para recorrer las filas y columnas de la matriz
    fila_actual = 0
    columna_actual = 0
    print(valor_unico)
    # Crear histograma solo para las columnas no seleccionadas
    for columna in columnas_analisis:
        if columna != columna_histograma:
            ax = axs[fila_actual, columna_actual]
            ax.hist(df_filtrado[columna], bins=50, alpha=0.5, label=columna, edgecolor='black', linewidth=2)

            # Configurar el título en dos líneas
            titulo = f"{valor_unico}\n{columna}"
            ax.set_title(titulo, fontsize=14)

            ax.set_xlabel("Valores", fontsize=12)
            ax.set_ylabel("Frecuencia", fontsize=12)
            ax.legend(fontsize=10)

            # Actualizar contadores de fila y columna
            columna_actual += 1
            if columna_actual == 5:
                columna_actual = 0
                fila_actual += 1

    # Ajustar el diseño de los subplots
    plt.tight_layout()
    plt.show()


# CREAR QQ PLOT CON DATA OUTLIER

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# Paso 1: Importar un dataframe en csv
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/8_DOM_OUT5.csv'  # Reemplaza 'tu_archivo.csv' con el nombre de tu archivo CSV
df = pd.read_csv(archivo_csv)

# Paso 2: Seleccionar desde qué columna se iniciará el análisis
try:
    inicio_columna = int(input("Ingrese el número de la columna desde la cual se iniciará el análisis: "))
except ValueError:
    print("Por favor, ingrese un número válido.")

# Validar que el número de columna ingresado sea válido
if inicio_columna < 0 or inicio_columna >= len(df.columns):
    print("Número de columna no válido. Saliendo.")
    exit()

# Filtrar las columnas a partir del inicio_columna
columnas_analisis = df.columns[inicio_columna:]

# Paso 3: Seleccionar una columna para hacer Q-Q plots
columna_qqplot = input("Ingrese el nombre de la columna para hacer Q-Q plots: ")

# Validar que la columna seleccionada exista en el dataframe
if columna_qqplot not in df.columns:
    print("La columna seleccionada no existe en el dataframe. Saliendo.")
    exit()

# Obtener valores únicos de la columna para Q-Q plots
valores_unicos = df[columna_qqplot].unique()

# Paso 4: Graficar Q-Q plots solo para las columnas no seleccionadas
for valor_unico in valores_unicos:
    # Filtrar el dataframe por el valor único en la columna para Q-Q plots
    df_filtrado = df[df[columna_qqplot] == valor_unico]

    # Crear subplots para cada valor único
    fig, axs = plt.subplots(5, 5, figsize=(25, 25))

    # Contadores para recorrer las filas y columnas de la matriz
    fila_actual = 0
    columna_actual = 0
    print(valor_unico)
    # Crear Q-Q plot solo para las columnas no seleccionadas
    for columna in columnas_analisis:
        if columna != columna_qqplot:
            ax = axs[fila_actual, columna_actual]
            stats.probplot(df_filtrado[columna], plot=ax, fit=True)

            # Configurar el título en dos líneas
            titulo = f"{valor_unico}\n{columna}"
            ax.set_title(titulo, fontsize=14)

            # Actualizar contadores de fila y columna
            columna_actual += 1
            if columna_actual == 5:
                columna_actual = 0
                fila_actual += 1

    # Ajustar el diseño de los subplots
    plt.tight_layout()
    plt.show()


# CREAR GRÁFICO DE CAJAS Y BIGOTES

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Paso 1: Importar un dataframe en csv
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/8_DOM_OUT5.csv'  # Reemplaza 'tu_archivo.csv' con el nombre de tu archivo CSV
df = pd.read_csv(archivo_csv)

# Paso 2: Seleccionar desde qué columna se iniciará el análisis
try:
    inicio_columna = int(input("Ingrese el número de la columna desde la cual se iniciará el análisis: "))
except ValueError:
    print("Por favor, ingrese un número válido.")

# Validar que el número de columna ingresado sea válido
if inicio_columna < 0 or inicio_columna >= len(df.columns):
    print("Número de columna no válido. Saliendo.")
    exit()

# Filtrar las columnas a partir del inicio_columna
columnas_analisis = df.columns[inicio_columna:]

# Paso 3: Seleccionar una columna para hacer box plot (eje y)
columna_boxplot_y = input("Ingrese el nombre de la columna cuyos valores únicos servirán como eje Y en el boxplot: ")

# Validar que la columna seleccionada exista en el dataframe
if columna_boxplot_y not in df.columns:
    print("La columna seleccionada no existe en el dataframe. Saliendo.")
    exit()

# Paso 4: Crear una lista con las columnas restantes
columnas_restantes = list(columnas_analisis[columnas_analisis != columna_boxplot_y])

# Paso 5: Crear el boxplot utilizando Seaborn
for i in range(len(columnas_restantes)):
    plt.figure(figsize=(12, 6))
    sns.boxplot(x=columnas_restantes[i], y=columna_boxplot_y, data=df)

# Configurar el título y etiquetas de ejes
    plt.xlabel(columnas_restantes[i], fontsize=14)
    plt.ylabel(columna_boxplot_y, fontsize=14)

# Mostrar el gráfico
    plt.show()


# 8. REMOCIÓN DE DATA OUTLIER 1
* Se utilizaron los valores extremos de la caja y bigotes

In [ ]:
'C:/10mo/1_TESIS 2/1_GEOQUIMICA/1_DOMINIOS/EXCEL/7_DOM_PRESEP.csv'

In [ ]:
import pandas as pd

# 1. Importar un dataframe en csv
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/7_DOM_PRESEP.csv'
df = pd.read_csv(archivo_csv)
decimal=int(input("Ingrese el número de decimales: "))
# 2. Pide el número de columna desde el cual se realizará el análisis
num_columna_inicial = int(input(f"Ingrese el número de columna desde 0 hasta {len(df.columns)-1} para iniciar el análisis: "))
columna_inicial = df.columns[num_columna_inicial]

# 3. Pide el nombre de la columna para obtener los valores únicos
columna_valores_unicos = input("Ingrese el nombre de la columna para obtener los valores únicos: ")

# 4. Inicializar diccionario para almacenar resumen de valores eliminados por columna y valor único
resumen_valores_eliminados = {}

# 5. Obtener valores de rango intercuartil, cuartil 1 y cuartil 3 para las columnas restantes
for columna in df.columns[num_columna_inicial:]:
    if columna != columna_valores_unicos:
        resumen_valores_eliminados[columna] = {"valor_unico": [], "cantidad": [], "porcentaje": []}

        for valor_unico in df[columna_valores_unicos].unique():
            q1 = round(df[(df[columna_valores_unicos] == valor_unico) & (df[columna].notnull())][columna].quantile(0.25), decimal)
            q3 = round(df[(df[columna_valores_unicos] == valor_unico) & (df[columna].notnull())][columna].quantile(0.75), decimal)
            iqr = round(q3 - q1, decimal)

            # 6. Eliminar del dataframe los valores fuera de los umbrales definidos
            valores_menor = round((q1 - 1.5 * iqr), decimal)
            valores_mayor = round((q3 + 1.5 * iqr), decimal)

            filtro = (df[columna_valores_unicos] == valor_unico) & ((df[columna] < valores_menor) | (df[columna] > valores_mayor))

            # Reemplazar los valores fuera de los umbrales con NaN
            df.loc[filtro, columna] = None

            cantidad_eliminada = filtro.sum()
            porcentaje_eliminado = (cantidad_eliminada / df[(df[columna_valores_unicos] == valor_unico) & (df[columna].notnull())][columna].count()) * 100

            resumen_valores_eliminados[columna]["valor_unico"].append(valor_unico)
            resumen_valores_eliminados[columna]["cantidad"].append(cantidad_eliminada)
            resumen_valores_eliminados[columna]["porcentaje"].append(round(porcentaje_eliminado, 2))

# 7. Mostrar resumen de la cantidad de valores eliminados y su porcentaje por columna y valor único
print("\nResumen de valores eliminados por columna y valor único:")
for columna, valores_eliminados in resumen_valores_eliminados.items():
    print(f"\nColumna: {columna}")
    for valor_unico, cantidad, porcentaje in zip(valores_eliminados["valor_unico"], valores_eliminados["cantidad"], valores_eliminados["porcentaje"]):
        print(f"Dominio: {valor_unico} - Cantidad Eliminada: {cantidad} valores ({porcentaje}%)")

# 8. Exportar el dataframe modificado a un nuevo archivo CSV
archivo_salida = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/8_DOM_OUT1.csv'  # Nombre del nuevo archivo CSV
df.round(decimal).to_csv(archivo_salida, index=False)


# 8.1 REMOCIÓN DE DATA OUTLIER 2
* En caso se quiera remover outliers por dominio individual

In [ ]:
import pandas as pd

# 1. Importar un dataframe en csv
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/8_DOM_OUT4.csv'
df = pd.read_csv(archivo_csv)
decimal=int(input("Ingrese el número de decimales: "))
# 2. Pide el número de columna desde el cual se realizará el análisis
num_columna_inicial = int(input(f"Ingrese el número de columna desde 0 hasta {len(df.columns)-1} para iniciar el análisis: "))
columna_inicial = df.columns[num_columna_inicial]

# 3. Pide el nombre de la columna para obtener los valores únicos
columna_valores_unicos = input("Ingrese el nombre de la columna para obtener los valores únicos: ")

# 4. Inicializar diccionario para almacenar resumen de valores eliminados por columna y valor único
resumen_valores_eliminados = {}

# 5. Obtener valores de rango intercuartil, cuartil 1 y cuartil 3 para las columnas restantes
for columna in df.columns[num_columna_inicial:]:
    if columna != columna_valores_unicos:
        resumen_valores_eliminados[columna] = {"valor_unico": [], "cantidad": [], "porcentaje": []}

        for valor_unico in df[columna_valores_unicos].unique():
            q1 = round(df[(df[columna_valores_unicos] == valor_unico) & (df[columna].notnull())][columna].quantile(0.25), decimal)
            q3 = round(df[(df[columna_valores_unicos] == valor_unico) & (df[columna].notnull())][columna].quantile(0.75), decimal)
            iqr = round(q3 - q1, decimal)

            # 6. Eliminar del dataframe los valores fuera de los umbrales definidos
            valores_menor = round((q1 - 1.5 * iqr), decimal)
            valores_mayor = round((q3 + 1.5 * iqr), decimal)

            filtro = (df[columna_valores_unicos] == valor_unico) & ((df[columna] < valores_menor) | (df[columna] > valores_mayor))

            # Reemplazar los valores fuera de los umbrales con NaN
            df.loc[filtro, columna] = None

            cantidad_eliminada = filtro.sum()
            porcentaje_eliminado = (cantidad_eliminada / df[(df[columna_valores_unicos] == valor_unico) & (df[columna].notnull())][columna].count()) * 100

            resumen_valores_eliminados[columna]["valor_unico"].append(valor_unico)
            resumen_valores_eliminados[columna]["cantidad"].append(cantidad_eliminada)
            resumen_valores_eliminados[columna]["porcentaje"].append(round(porcentaje_eliminado, 2))

# 7. Mostrar resumen de la cantidad de valores eliminados y su porcentaje por columna y valor único
print("\nResumen de valores eliminados por columna y valor único:")
for columna, valores_eliminados in resumen_valores_eliminados.items():
    print(f"\nColumna: {columna}")
    for valor_unico, cantidad, porcentaje in zip(valores_eliminados["valor_unico"], valores_eliminados["cantidad"], valores_eliminados["porcentaje"]):
        print(f"Dominio: {valor_unico} - Cantidad Eliminada: {cantidad} valores ({porcentaje}%)")

# 8. Exportar el dataframe modificado a un nuevo archivo CSV
archivo_salida = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/8_DOM_OUT5.csv'  # Nombre del nuevo archivo CSV
df.round(decimal).to_csv(archivo_salida, index=False)


# 9. SEPARAR CSV SEGÚN LOS DOMINIOS
* De aquí en adelante, los códigos se aplicarán por cada dominio indivdual; por ello, es necesario separar en diferentes CSV por cada dominio

In [ ]:
import pandas as pd

# Paso 1: Importa un dataframe en csv
archivo_csv = 'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/8_DOM_OUT5.csv'  # Reemplaza 'tu_archivo.csv' con el nombre de tu archivo CSV
df = pd.read_csv(archivo_csv)
decimal=int(input("Ingrese el número de decimales: "))
# Paso 2: Seleccionar una columna
columna_seleccionada123 = input("Ingrese el nombre de la columna que desea seleccionar: ")

# Verificar que la columna seleccionada existe en el dataframe
if columna_seleccionada123 not in df.columns:
    print("La columna seleccionada no existe en el dataframe. Saliendo.")
    exit()

# Obtener valores únicos de la columna seleccionada
valores_unicos123 = df[columna_seleccionada123].unique()

# Exportar nuevos CSV en función a los valores únicos de la columna
for valor123 in valores_unicos123:
    # Filtrar el dataframe por el valor único
    df_filtrado = df[df[columna_seleccionada123] == valor123]

    # Generar nombre para el nuevo archivo CSV
    nombre_archivo_salida = f"9_{columna_seleccionada123}_{str(valor123)}.csv"

    # Exportar el dataframe filtrado a un nuevo archivo CSV
    df_filtrado.to_csv(f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/{nombre_archivo_salida}", index=False)

    print(f"Archivo '{nombre_archivo_salida}' exportado para el valor '{valor123}'.")


# 10. Z SCORE Y PRUEBAS DE NORMALIDAD
* Se debe aplicar la prueba por cada csv de dominio por separado
* Se crean nuevas columnas con valores Zscore
* Se aplica las pruebas de Kolmogorov (cantidad de muestras mayor a 30) y Shapiro (cantidad de muestras menor a 30)
* Permite añadir otras columnas que no hayan pasado el criterio de ser mayor a 0.05 el p_value
* Se crea un dataframe llamado dfZ que contiene los valores Z para usarlos en gráficos Q-Q plot y ver si hay elementos que cumplen con la recta

## 10.1. LISTA DE NOMBRES
* Se obtiene una lista que contiene los nombres de cada dominio
* Esto con el fin de reemplazar el nombre del archivo en la ruta de importación
* No modificar lo que está antes de la variable {b} en el resto del código

In [ ]:
# NO MODIFICAR
valores_unicos123 = df[columna_seleccionada123].unique()
nombres_rutas123=[]
nombres_rutas_corchetes123=[]
for a in valores_unicos123:
    nombres_rutas123.append(a)
    nombres_rutas_corchetes123.append(a)
print(nombres_rutas123)
for b in nombres_rutas123:
    print(b)
for c in nombres_rutas_corchetes123:
    print(c)

### VERSIÓN AUTOMÁTICA
* Importa todos los archivos automáticamente
* Si te equivocas en insertar los parámetros, no se puede corregir
* En caso falten algunos archivos y no quieras volver a escribir todo, hay una versión individual

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from scipy.stats import kstest, shapiro

for b in nombres_rutas123:

    # 1. Importar un CSV
    archivo_csv = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/9_DOM_{b}.csv"
    df1 = pd.read_csv(archivo_csv)
    print(f"Se está leyendo el archivo {archivo_csv}")
    df = df1.copy()
    decimal = int(input("Ingrese el número de decimales: "))
    dom = input("Ingrese el nombre de columna de sus dominios: ")
    valores_unicos1 = df1[dom].unique()
    # 2. Pedir el número de la columna para iniciar el análisis (excluyendo la última columna)
    num_columna_inicio = int(input("Ingrese el número de la columna a partir del cual iniciará el análisis: "))
    elementos_noZ = df[df.columns[:num_columna_inicio]]  # Excluyendo la última columna
    elementos_noZ2 = df[df.columns[num_columna_inicio: - 1]]
    
    # 3. Por cada columna, transformar a z score y copiar en una nueva columna
    for i in range(num_columna_inicio, len(df.columns) - 1):  # Excluyendo la última columna
        columna_actual = df.columns[i]
        z_scores = (df[columna_actual] - np.mean(df[columna_actual])) / np.std(df[columna_actual])
        nombre_nueva_columna = "Z_" + columna_actual
        df[nombre_nueva_columna] = z_scores
    
    print(df)
    dfZ = df.copy()
    # 5. Solo a las columnas cuyo nombre inicie con "Z_", aplicar la prueba de normalidad
    cumplieron7 = []
    for columna in df.columns:
        if columna.startswith("Z_"):
            num_values = df[columna].dropna()
            print(columna)
            if len(num_values) > 30:
                # Aplicar prueba Kolmogorov-Smirnov-Lilliefors
                ks_statistic, ks_p_value = sm.stats.diagnostic.lilliefors(num_values, 'norm', pvalmethod='approx')
                prueba_nombre = 'Kolmogorov-Smirnov-Lilliefors'
            else:
                # Aplicar prueba Shapiro-Wilk
                shapiro_statistic, shapiro_p_value = shapiro(num_values)
                ks_statistic, ks_p_value = shapiro_statistic, shapiro_p_value  # Para mostrar en la salida
                prueba_nombre = 'Shapiro-Wilk'
    
            # 6. Mostrar la columna con su respectivo p value
            print(f"\nPrueba de normalidad para {columna} utilizando {prueba_nombre}:")
            print(f"{prueba_nombre} Statistic: {ks_statistic}")
            print(f"P Value: {ks_p_value:.3f}")
    
            # 7. Agrupar las columnas cuyos p value sean mayores a 0.05
            if ks_p_value > 0.05:
                cumplieron7.append((columna, prueba_nombre, round(ks_p_value, 3)))
    
    print("Elementos que pasaron la condición >0.05")
    for i in range(len(cumplieron7)):
        print(f"{i + 1}. {cumplieron7[i][0][2:]} - {cumplieron7[i][1]} - p_value: {cumplieron7[i][2]}")
    
    # 8. Cambiar los nombres de columna de la lista Cumplieron7 quitándoles el prefijo "Z_"
    for i in range(len(cumplieron7)):
        cumplieron7[i] = (cumplieron7[i][0][2:], cumplieron7[i][1], cumplieron7[i][2])
    
    # 9. Preguntar por las columnas extras
    print("\nColumnas disponibles:")
    for i, col in enumerate(elementos_noZ2.columns, 1):
        print(f"{i}. {col}")
    extras = str(input("¿Quiere agregar columnas extras normales (S/N)?: "))
    if extras == "S":
        columnas_extras_indices = input("\nIngrese los números de las columnas extras separados por comas (sin espacios): ").split(',')
        columnas_extras = [elementos_noZ2.columns[int(idx) - 1] for idx in columnas_extras_indices]
    else:
        columnas_extras = []
    
    # 10. Comparar los nombres de columna de Cumplieron7 con los nombres de columnas de Elementos_noZ y columnas extras
    sicumple = [col for col in elementos_noZ2.columns if col in [c[0] for c in cumplieron7] + columnas_extras]
    
    # 11. En un nuevo DataFrame, unir las columnas que cumplen el criterio
    df_cumplen_criterio = pd.concat([elementos_noZ, df[sicumple], df1[dom]], axis=1)
    
    # 12. En un nuevo DataFrame, unir las columnas que NO cumplen el criterio
    df_no_cumplen_criterio = df.loc[:, ~df.columns.isin(sicumple)]
    
    # Eliminar las columnas que comienzan con "Z_" del DataFrame df_no_cumplen_criterio
    df_no_cumplen_criterio = df_no_cumplen_criterio.loc[:, ~df_no_cumplen_criterio.columns.str.startswith("Z_")]
    
    # Imprimir los DataFrames resultantes
    print("\nDataFrame de columnas que cumplen el criterio:")
    print(df_cumplen_criterio)
    
    print("\nDataFrame de columnas que NO cumplen el criterio (sin columnas que comienzan con 'Z_'):")
    print(df_no_cumplen_criterio)
    
    # 13. Exportar los DataFrames resultantes a CSV
    archivo_salida_cumplen = f'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/10_{valores_unicos1}_NML.csv'
    archivo_salida_no_cumplen = f'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/10_{valores_unicos1}_NPARA.csv'
    
    df_cumplen_criterio.round(decimal).to_csv(archivo_salida_cumplen, index=False)
    df_no_cumplen_criterio.round(decimal).to_csv(archivo_salida_no_cumplen, index=False)
    
    print(f"\nLos DataFrames resultantes han sido exportados a:\n- {archivo_salida_cumplen}\n- {archivo_salida_no_cumplen}.")
    

### VERSIÓN INDIVIDUAL
* Acá solo debes modificar la ruta del archivo

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from scipy.stats import kstest, shapiro

# 1. Importar un CSV
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/9_DOM_NO.csv"
df1 = pd.read_csv(archivo_csv)
df = df1.copy()
decimal = int(input("Ingrese el número de decimales: "))
dom = input("Ingrese el nombre de columna de sus dominios: ")
valores_unicos1 = df1[dom].unique()
# 2. Pedir el número de la columna para iniciar el análisis (excluyendo la última columna)
num_columna_inicio = int(input("Ingrese el número de la columna a partir del cual iniciará el análisis: "))
elementos_noZ = df[df.columns[:num_columna_inicio]]  # Excluyendo la última columna
elementos_noZ2 = df[df.columns[num_columna_inicio: - 1]]

# 3. Por cada columna, transformar a z score y copiar en una nueva columna
for i in range(num_columna_inicio, len(df.columns) - 1):  # Excluyendo la última columna
    columna_actual = df.columns[i]
    z_scores = (df[columna_actual] - np.mean(df[columna_actual])) / np.std(df[columna_actual])
    nombre_nueva_columna = "Z_" + columna_actual
    df[nombre_nueva_columna] = z_scores

print(df)
dfZ = df.copy()
# 5. Solo a las columnas cuyo nombre inicie con "Z_", aplicar la prueba de normalidad
cumplieron7 = []
for columna in df.columns:
    if columna.startswith("Z_"):
        num_values = df[columna].dropna()
        print(columna)
        if len(num_values) > 30:
            # Aplicar prueba Kolmogorov-Smirnov-Lilliefors
            ks_statistic, ks_p_value = sm.stats.diagnostic.lilliefors(num_values, 'norm', pvalmethod='approx')
            prueba_nombre = 'Kolmogorov-Smirnov-Lilliefors'
        else:
            # Aplicar prueba Shapiro-Wilk
            shapiro_statistic, shapiro_p_value = shapiro(num_values)
            ks_statistic, ks_p_value = shapiro_statistic, shapiro_p_value  # Para mostrar en la salida
            prueba_nombre = 'Shapiro-Wilk'

        # 6. Mostrar la columna con su respectivo p value
        print(f"\nPrueba de normalidad para {columna} utilizando {prueba_nombre}:")
        print(f"{prueba_nombre} Statistic: {ks_statistic}")
        print(f"P Value: {ks_p_value:.3f}")

        # 7. Agrupar las columnas cuyos p value sean mayores a 0.05
        if ks_p_value > 0.05:
            cumplieron7.append((columna, prueba_nombre, round(ks_p_value, 3)))

print("Elementos que pasaron la condición >0.05")
for i in range(len(cumplieron7)):
    print(f"{i + 1}. {cumplieron7[i][0][2:]} - {cumplieron7[i][1]} - p_value: {cumplieron7[i][2]}")

# 8. Cambiar los nombres de columna de la lista Cumplieron7 quitándoles el prefijo "Z_"
for i in range(len(cumplieron7)):
    cumplieron7[i] = (cumplieron7[i][0][2:], cumplieron7[i][1], cumplieron7[i][2])

# 9. Preguntar por las columnas extras
print("\nColumnas disponibles:")
for i, col in enumerate(elementos_noZ2.columns, 1):
    print(f"{i}. {col}")
extras = str(input("¿Quiere agregar columnas extras normales (S/N)?: "))
if extras == "S":
    columnas_extras_indices = input("\nIngrese los números de las columnas extras separados por comas (sin espacios): ").split(',')
    columnas_extras = [elementos_noZ2.columns[int(idx) - 1] for idx in columnas_extras_indices]
else:
    columnas_extras = []

# 10. Comparar los nombres de columna de Cumplieron7 con los nombres de columnas de Elementos_noZ y columnas extras
sicumple = [col for col in elementos_noZ2.columns if col in [c[0] for c in cumplieron7] + columnas_extras]

# 11. En un nuevo DataFrame, unir las columnas que cumplen el criterio
df_cumplen_criterio = pd.concat([elementos_noZ, df[sicumple], df1[dom]], axis=1)

# 12. En un nuevo DataFrame, unir las columnas que NO cumplen el criterio
df_no_cumplen_criterio = df.loc[:, ~df.columns.isin(sicumple)]

# Eliminar las columnas que comienzan con "Z_" del DataFrame df_no_cumplen_criterio
df_no_cumplen_criterio = df_no_cumplen_criterio.loc[:, ~df_no_cumplen_criterio.columns.str.startswith("Z_")]

# Imprimir los DataFrames resultantes
print("\nDataFrame de columnas que cumplen el criterio:")
print(df_cumplen_criterio)

print("\nDataFrame de columnas que NO cumplen el criterio (sin columnas que comienzan con 'Z_'):")
print(df_no_cumplen_criterio)

# 13. Exportar los DataFrames resultantes a CSV
archivo_salida_cumplen = f'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/10_{valores_unicos1}_NML.csv'
archivo_salida_no_cumplen = f'C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_UNIDO/EXCEL/10_{valores_unicos1}_NPARA.csv'

df_cumplen_criterio.round(decimal).to_csv(archivo_salida_cumplen, index=False)
df_no_cumplen_criterio.round(decimal).to_csv(archivo_salida_no_cumplen, index=False)

print(f"\nLos DataFrames resultantes han sido exportados a:\n- {archivo_salida_cumplen}\n- {archivo_salida_no_cumplen}.")


# GRÁFICO Q-Q PLOT A Z VALUES
* Se armarán gráficos Q-Q plot a las columnas de Z values para ver si se ajustan a la recta

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

print("Índices de columnas disponibles:")
for i, columna1 in enumerate(dfZ.columns):
    print(f"{i + 1}. {columna1}")

# Paso 2: Seleccionar desde qué columna se iniciará el análisis
try:
    inicio_columna = int(input("Ingrese el número de la columna desde la cual se iniciará el análisis: "))
except ValueError:
    print("Por favor, ingrese un número válido.")

# Validar que el número de columna ingresado sea válido
if inicio_columna < 0 or inicio_columna >= len(dfZ.columns):
    print("Número de columna no válido. Saliendo.")
    exit()

# Filtrar las columnas a partir del inicio_columna
columnas_analisis = dfZ.columns[inicio_columna:]

# Paso 3: Seleccionar una columna para hacer Q-Q plots
columna_qqplot = input("Ingrese el nombre de la columna de dominios para hacer Q-Q plots: ")
valores_unicos = dfZ[columna_qqplot].unique()
    # Crear subplots para cada valor único
fig, axs = plt.subplots(6, 6, figsize=(25, 25))

    # Contadores para recorrer las filas y columnas de la matriz
fila_actual = 0
columna_actual = 0

    # Crear Q-Q plot solo para las columnas no seleccionadas
for columna in columnas_analisis:
    if columna != columna_qqplot:
        ax = axs[fila_actual, columna_actual]
        stats.probplot(dfZ[columna], plot=ax, fit=True)

        # Configurar el título en dos líneas
        titulo = f"{valores_unicos}\n{columna}"
        ax.set_title(titulo, fontsize=14)        
        # Actualizar contadores de fila y columna
        columna_actual += 1
        if columna_actual == 6:
            columna_actual = 0
            fila_actual += 1

    # Ajustar el diseño de los subplots
plt.tight_layout()
plt.show()


# 11.1. CONVERSIÓN LN - PARAMÉTRICOS
* Se debe hacer la conversión tanto para los paramétricos como no paramétricos

### VERSIÓN AUTOMÁTICA

In [ ]:
import pandas as pd
import math

for c in nombres_rutas_corchetes123:
    # 1. Importa un CSV
    archivo_csv = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/10_['{c}']_NML.csv"
    df = pd.read_csv(archivo_csv)
    
    # 2. Pide el número de la columna para iniciar el análisis
    inicio_analisis = int(input("Ingrese el número de la columna para iniciar el análisis: "))
    dom = input("Ingrese el nombre de columna de sus dominios: ")
    valores_unicos2 = df[dom].unique()
    # 3. Modificar columnas que comienzan con "Ln"
    for i in range(inicio_analisis, df.shape[1] - 1):
        if df.columns[i].startswith("Ln"):
            # Agregar "C" al inicio del nombre de la columna
            nuevo_nombre_columna = "C" + df.columns[i]
            df.rename(columns={df.columns[i]: nuevo_nombre_columna}, inplace=True)
    
            # Aplicar la función exponencial natural a los valores no vacíos y no nulos
            df[nuevo_nombre_columna] = df[nuevo_nombre_columna].apply(lambda x: round(math.exp(float(x)), 2) if pd.notnull(x) and x != 0 else x)
    
    # 4. Mostrar el DataFrame resultante
    print("\nDataFrame resultante:")
    print(df)
    
    # 5. Exportar el DataFrame resultante
    nombre_archivo_salida = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/11_{valores_unicos2}_NOLN_PARA.csv" 
    df.to_csv(nombre_archivo_salida, index=False)
    
    print(f"\nEl DataFrame resultante ha sido exportado a {nombre_archivo_salida}.")


### VERSIÓN INDIVIDUAL

In [ ]:
import pandas as pd
import math

# 1. Importa un CSV
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/10_['Volcanosedimentario Terciario']_NML.csv"
df = pd.read_csv(archivo_csv)

# 2. Pide el número de la columna para iniciar el análisis
inicio_analisis = int(input("Ingrese el número de la columna para iniciar el análisis: "))
dom = input("Ingrese el nombre de columna de sus dominios: ")
valores_unicos2 = df[dom].unique()
# 3. Modificar columnas que comienzan con "Ln"
for i in range(inicio_analisis, df.shape[1] - 1):
    if df.columns[i].startswith("Ln"):
        # Agregar "C" al inicio del nombre de la columna
        nuevo_nombre_columna = "C" + df.columns[i]
        df.rename(columns={df.columns[i]: nuevo_nombre_columna}, inplace=True)

        # Aplicar la función exponencial natural a los valores no vacíos y no nulos
        df[nuevo_nombre_columna] = df[nuevo_nombre_columna].apply(lambda x: round(math.exp(float(x)), 2) if pd.notnull(x) and x != 0 else x)

# 4. Mostrar el DataFrame resultante
print("\nDataFrame resultante:")
print(df)

# 5. Exportar el DataFrame resultante
nombre_archivo_salida = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/11_{valores_unicos2}_NOLN_PARA.csv" 
df.to_csv(nombre_archivo_salida, index=False)

print(f"\nEl DataFrame resultante ha sido exportado a {nombre_archivo_salida}.")


# 11.2. CONVERSIÓN LN - NO PARAMÉTRICOS

### VERSIÓN AUTOMÁTICA

In [ ]:
import pandas as pd
import math

for c in nombres_rutas_corchetes123:

    # 1. Importa un CSV
    archivo_csv = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/10_['{c}']_NPARA.csv"
    df = pd.read_csv(archivo_csv)
    
    # 2. Pide el número de la columna para iniciar el análisis
    inicio_analisis = int(input("Ingrese el número de la columna para iniciar el análisis: "))
    dom = input("Ingrese el nombre de columna de sus dominios: ")
    valores_unicos2 = df[dom].unique()
    # 3. Modificar columnas que comienzan con "Ln"
    for i in range(inicio_analisis, df.shape[1] - 1):
        if df.columns[i].startswith("Ln"):
            # Agregar "C" al inicio del nombre de la columna
            nuevo_nombre_columna = "C" + df.columns[i]
            df.rename(columns={df.columns[i]: nuevo_nombre_columna}, inplace=True)
    
            # Aplicar la función exponencial natural a los valores no vacíos y no nulos
            df[nuevo_nombre_columna] = df[nuevo_nombre_columna].apply(lambda x: round(math.exp(float(x)), 2) if pd.notnull(x) and x != 0 else x)
    
    # 4. Mostrar el DataFrame resultante
    print("\nDataFrame resultante:")
    print(df)
    
    # 5. Exportar el DataFrame resultante
    nombre_archivo_salida = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/11_{valores_unicos2}_NOLN_NPARA.csv" 
    df.to_csv(nombre_archivo_salida, index=False)
    
    print(f"\nEl DataFrame resultante ha sido exportado a {nombre_archivo_salida}.")

### VERSION INDIVIDUAL

In [ ]:
import pandas as pd
import math

# 1. Importa un CSV
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/10_['Volcanosedimentario Terciario']_NPARA.csv"
df = pd.read_csv(archivo_csv)

# 2. Pide el número de la columna para iniciar el análisis
inicio_analisis = int(input("Ingrese el número de la columna para iniciar el análisis: "))
dom = input("Ingrese el nombre de columna de sus dominios: ")
valores_unicos2 = df[dom].unique()
# 3. Modificar columnas que comienzan con "Ln"
for i in range(inicio_analisis, df.shape[1] - 1):
    if df.columns[i].startswith("Ln"):
        # Agregar "C" al inicio del nombre de la columna
        nuevo_nombre_columna = "C" + df.columns[i]
        df.rename(columns={df.columns[i]: nuevo_nombre_columna}, inplace=True)

        # Aplicar la función exponencial natural a los valores no vacíos y no nulos
        df[nuevo_nombre_columna] = df[nuevo_nombre_columna].apply(lambda x: round(math.exp(float(x)), 2) if pd.notnull(x) and x != 0 else x)

# 4. Mostrar el DataFrame resultante
print("\nDataFrame resultante:")
print(df)

# 5. Exportar el DataFrame resultante
nombre_archivo_salida = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/11_{valores_unicos2}_NOLN_NPARA.csv" 
df.to_csv(nombre_archivo_salida, index=False)

print(f"\nEl DataFrame resultante ha sido exportado a {nombre_archivo_salida}.")

# 12.1. PARÁMETROS UNIVARIADOS NORMALES

### VERSIÓN AUTOMÁTICA

In [ ]:
## 
import pandas as pd

for c in nombres_rutas_corchetes123:

    # 1. Importa un CSV
    archivo_csv = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/11_['{c}']_NOLN_PARA.csv" 
    df = pd.read_csv(archivo_csv)
    
    # 2. Pide el número de la columna para iniciar el análisis
    inicio_analisis = int(input("Ingrese el número de la columna para iniciar el análisis: "))
    dom = input("Ingrese el nombre de columna de sus dominios: ")
    valores_unicos3 = df[dom].unique()
    # 3. Calcular media y desviación estándar para cada columna
    resultados = []
    nombres_columnas = []
    for i in range(inicio_analisis, df.shape[1] - 1):
        columna = df.iloc[:, i]
        media = round(columna.mean(), 2)
        desviacion_estandar = round(columna.std(), 2)
        resultados.append((media, desviacion_estandar))
        nombres_columnas.append(df.columns[i])
    
    # 4. Calcular VF, T1, T2 y T3 por cada columna
    resultados_vf_t1_t2_t3 = []
    for i, (media, desviacion_estandar) in enumerate(resultados):
        vf = media
        t1 = media + desviacion_estandar
        t2 = media + 2 * desviacion_estandar
        t3 = media + 3 * desviacion_estandar
        resultados_vf_t1_t2_t3.append((vf, t1, t2, t3))
    
    # 5. Mostrar VF, T1, T2 y T3 por cada columna con el nombre de la columna
    print("\nResultados:")
    for i, (vf, t1, t2, t3) in enumerate(resultados_vf_t1_t2_t3):
        nombre_columna = nombres_columnas[i]
        print(f"\nColumna {nombre_columna} ({inicio_analisis + i + 1}):")
        print(f"VF: {vf}, T1: {t1}, T2: {t2}, T3: {t3}")
    
    # 6. Copiar cada columna y aplicar las condiciones
    for i, (vf, t1, t2, t3) in enumerate(resultados_vf_t1_t2_t3):
        nombre_columna_copia = f"ANM_{nombres_columnas[i]}"
        df[nombre_columna_copia] = df.iloc[:, inicio_analisis + i].apply(lambda x: "Valor de fondo" if vf <= x < t1 else ("Anomalia baja" if t1 <= x < t2 else ("Anomalia media" if t2 <= x < t3 else ("Anomalia alta" if t3 <= x else "Muestra"))))
    
    # Mostrar el DataFrame resultante
    print("\nDataFrame resultante:")
    print(df)
    
    nombre_archivo_salida = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/12_{valores_unicos3}_UNIV_PARA.csv" 
    df.to_csv(nombre_archivo_salida, index=False)

### VERSIÓN INDIVIDUAL

In [ ]:
import pandas as pd

# 1. Importa un CSV
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/11_['NO']_NOLN_PARA.csv" 
df = pd.read_csv(archivo_csv)

# 2. Pide el número de la columna para iniciar el análisis
inicio_analisis = int(input("Ingrese el número de la columna para iniciar el análisis: "))
dom = input("Ingrese el nombre de columna de sus dominios: ")
valores_unicos3 = df[dom].unique()
# 3. Calcular media y desviación estándar para cada columna
resultados = []
nombres_columnas = []
for i in range(inicio_analisis, df.shape[1] - 1):
    columna = df.iloc[:, i]
    media = round(columna.mean(), 2)
    desviacion_estandar = round(columna.std(), 2)
    resultados.append((media, desviacion_estandar))
    nombres_columnas.append(df.columns[i])

# 4. Calcular VF, T1, T2 y T3 por cada columna
resultados_vf_t1_t2_t3 = []
for i, (media, desviacion_estandar) in enumerate(resultados):
    vf = media
    t1 = media + desviacion_estandar
    t2 = media + 2 * desviacion_estandar
    t3 = media + 3 * desviacion_estandar
    resultados_vf_t1_t2_t3.append((vf, t1, t2, t3))

# 5. Mostrar VF, T1, T2 y T3 por cada columna con el nombre de la columna
print("\nResultados:")
for i, (vf, t1, t2, t3) in enumerate(resultados_vf_t1_t2_t3):
    nombre_columna = nombres_columnas[i]
    print(f"\nColumna {nombre_columna} ({inicio_analisis + i + 1}):")
    print(f"VF: {vf}, T1: {t1}, T2: {t2}, T3: {t3}")

# 6. Copiar cada columna y aplicar las condiciones
for i, (vf, t1, t2, t3) in enumerate(resultados_vf_t1_t2_t3):
    nombre_columna_copia = f"ANM_{nombres_columnas[i]}"
    df[nombre_columna_copia] = df.iloc[:, inicio_analisis + i].apply(lambda x: "Valor de fondo" if vf <= x < t1 else ("Anomalia baja" if t1 <= x < t2 else ("Anomalia media" if t2 <= x < t3 else ("Anomalia alta" if t3 <= x else "Muestra"))))

# Mostrar el DataFrame resultante
print("\nDataFrame resultante:")
print(df)

nombre_archivo_salida = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/12_{valores_unicos3}_UNIV_PARA.csv" 
df.to_csv(nombre_archivo_salida, index=False)


# 12.2. PARÁMETROS UNIVARIADOS NO NORMALES

### VERSIÓN AUTOMÁTICA

In [ ]:
import pandas as pd

for c in nombres_rutas_corchetes123:

    # 1. Importa un CSV
    archivo_csv = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/11_['{c}']_NOLN_NPARA.csv" 
    df = pd.read_csv(archivo_csv)
    
    # 2. Pide el número de la columna para iniciar el análisis
    inicio_analisis = int(input("Ingrese el número de la columna para iniciar el análisis: "))
    dom = input("Ingrese el nombre de columna de sus dominios: ")
    valores_unicos3 = df[dom].unique()
    
    # 3. Calcular percentiles para VF, T1, T2 y T3 por cada columna
    resultados_percentiles = []
    nombres_columnas = []
    for i in range(inicio_analisis, df.shape[1] - 1):
        columna = df.iloc[:, i]
        vf = columna.median()  # Percentil 50 (mediana)
        t1 = columna.quantile(0.75)  # Percentil 75
        t2 = columna.quantile(0.95)  # Percentil 95
        t3 = columna.quantile(0.95) * 2  # Doble del Percentil 95
        resultados_percentiles.append((vf, t1, t2, t3))
        nombres_columnas.append(df.columns[i])
    
    # 4. Mostrar VF, T1, T2 y T3 por cada columna con el nombre de la columna
    print("\nResultados:")
    for i, (vf, t1, t2, t3) in enumerate(resultados_percentiles):
        nombre_columna = nombres_columnas[i]
        print(f"\nColumna {nombre_columna} ({inicio_analisis + i + 1}):")
        print(f"VF: {vf}, T1: {t1}, T2: {t2}, T3: {t3}")
    
    # 5. Copiar cada columna y aplicar las condiciones
    for i, (vf, t1, t2, t3) in enumerate(resultados_percentiles):
        nombre_columna_copia = f"ANM_{nombres_columnas[i]}"
        df[nombre_columna_copia] = df.iloc[:, inicio_analisis + i].apply(lambda x: "Valor de fondo" if vf <= x < t1 else ("Anomalia baja" if t1 <= x < t2 else ("Anomalia media" if t2 <= x < t3 else ("Anomalia alta" if t3 <= x else "Muestra"))))
    
    # 6. Mostrar el DataFrame resultante
    print("\nDataFrame resultante:")
    print(df)
    
    nombre_archivo_salida = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/12_{valores_unicos3}_UNIV_NPARA.csv" 
    df.to_csv(nombre_archivo_salida, index=False)

### VERSIÓN INDIVIDUAL

In [ ]:
import pandas as pd

# 1. Importa un CSV
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/11_['Volcanosedimentario Terciario']_NOLN_NPARA.csv" 
df = pd.read_csv(archivo_csv)

# 2. Pide el número de la columna para iniciar el análisis
inicio_analisis = int(input("Ingrese el número de la columna para iniciar el análisis: "))
dom = input("Ingrese el nombre de columna de sus dominios: ")
valores_unicos3 = df[dom].unique()

# 3. Calcular percentiles para VF, T1, T2 y T3 por cada columna
resultados_percentiles = []
nombres_columnas = []
for i in range(inicio_analisis, df.shape[1] - 1):
    columna = df.iloc[:, i]
    vf = columna.median()  # Percentil 50 (mediana)
    t1 = columna.quantile(0.75)  # Percentil 75
    t2 = columna.quantile(0.95)  # Percentil 95
    t3 = columna.quantile(0.95) * 2  # Doble del Percentil 95
    resultados_percentiles.append((vf, t1, t2, t3))
    nombres_columnas.append(df.columns[i])

# 4. Mostrar VF, T1, T2 y T3 por cada columna con el nombre de la columna
print("\nResultados:")
for i, (vf, t1, t2, t3) in enumerate(resultados_percentiles):
    nombre_columna = nombres_columnas[i]
    print(f"\nColumna {nombre_columna} ({inicio_analisis + i + 1}):")
    print(f"VF: {vf}, T1: {t1}, T2: {t2}, T3: {t3}")

# 5. Copiar cada columna y aplicar las condiciones
for i, (vf, t1, t2, t3) in enumerate(resultados_percentiles):
    nombre_columna_copia = f"ANM_{nombres_columnas[i]}"
    df[nombre_columna_copia] = df.iloc[:, inicio_analisis + i].apply(lambda x: "Valor de fondo" if vf <= x < t1 else ("Anomalia baja" if t1 <= x < t2 else ("Anomalia media" if t2 <= x < t3 else ("Anomalia alta" if t3 <= x else "Muestra"))))

# 6. Mostrar el DataFrame resultante
print("\nDataFrame resultante:")
print(df)

nombre_archivo_salida = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/12_{valores_unicos3}_UNIV_NPARA.csv" 
df.to_csv(nombre_archivo_salida, index=False)



# 13. CREAR Y EXPORTAR SHAPEFILES

In [ ]:
!pip install geopandas
!pip install fiona
!pip install shapely
!pip install pyproj
!pip install matplotlib

### VERSIÓN AUTOMÁTICA
* modificar la última parte del nombre
 * NPARA= No paramétrico
 * PARA= Paramétrico

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pyproj import CRS

for c in nombres_rutas_corchetes123:

    # 1. Importar un DataFrame desde un archivo CSV
    archivo_csv = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/EXCEL/12_['{c}']_UNIV_PARA.csv"  # Reemplaza con la ruta de tu archivo CSV
    df = pd.read_csv(archivo_csv)
    
    # 2. Muestra los índices de columnas y pide seleccionar una para ser las coordenadas X
    print("Índices de columnas disponibles:")
    for i, columna in enumerate(df.columns):
        print(f"{i + 1}. {columna}")
    
    indice_x = int(input("Seleccione el índice de la columna para las coordenadas X: ")) - 1
    columna_x = df.columns[indice_x]
    
    # 3. Muestra los índices de columnas y pide seleccionar una para ser las coordenadas Y
    print("\nÍndices de columnas disponibles:")
    for i, columna in enumerate(df.columns):
        print(f"{i + 1}. {columna}")
    
    indice_y = int(input("Seleccione el índice de la columna para las coordenadas Y: ")) - 1
    columna_y = df.columns[indice_y]
    
    dom = input("Ingrese el nombre de columna de sus dominios: ")
    nombre_dominio = df[dom].unique()
    
    # 4. Pide que escriba el nombre del dominio
    crs1=str(input("Ingrese el numero de sistema de coordenadas (32717: UTM 17S/ 32718: UTM 18S/ 32719: UTM 19S/4326: WGS84): "))
    crs= str("EPSG:"+ crs1)
    # 5. Crea un archivo Shapefile por cada columna que inicie con "ANM_"
    for columna in df.columns:
        if columna.startswith("ANM_"):
            nombre_shapefile = columna[4:] + "_" + nombre_dominio
    
            # Crear un GeoDataFrame con las coordenadas X e Y
            geometria = [Point(xy) for xy in zip(df[columna_x], df[columna_y])]
            gdf = gpd.GeoDataFrame(df, geometry=geometria, crs=crs)  # WGS84
    
            # Exportar el Shapefile
            ruta_shapefile = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/0_AG/SHP_P/{nombre_shapefile}.shp"  # Reemplaza con la ruta de salida
            gdf.to_file(ruta_shapefile)
    
            print(f"Shapefile '{nombre_shapefile}' exportado exitosamente en {ruta_shapefile}.")
    

### VERSION INDIVIDUAL

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pyproj import CRS

# 1. Importar un DataFrame desde un archivo CSV
archivo_csv = "C:/10mo/1_TESIS 2/1_GEOQUIMICA/2_NORMALIDAD/EXCEL/12_['Volcanosedimentario Terciario']_UNIV_NPARA.csv"  # Reemplaza con la ruta de tu archivo CSV
df = pd.read_csv(archivo_csv)

# 2. Muestra los índices de columnas y pide seleccionar una para ser las coordenadas X
print("Índices de columnas disponibles:")
for i, columna in enumerate(df.columns):
    print(f"{i + 1}. {columna}")

indice_x = int(input("Seleccione el índice de la columna para las coordenadas X: ")) - 1
columna_x = df.columns[indice_x]

# 3. Muestra los índices de columnas y pide seleccionar una para ser las coordenadas Y
print("\nÍndices de columnas disponibles:")
for i, columna in enumerate(df.columns):
    print(f"{i + 1}. {columna}")

indice_y = int(input("Seleccione el índice de la columna para las coordenadas Y: ")) - 1
columna_y = df.columns[indice_y]

dom = input("Ingrese el nombre de columna de sus dominios: ")
nombre_dominio = df[dom].unique()

# 4. Pide que escriba el nombre del dominio
crs1=str(input("Ingrese el numero de sistema de coordenadas (32717: UTM 17S/ 32718: UTM 18S/ 32719: UTM 19S/4326: WGS84): "))
crs= str("EPSG:"+ crs1)
# 5. Crea un archivo Shapefile por cada columna que inicie con "ANM_"
for columna in df.columns:
    if columna.startswith("ANM_"):
        nombre_shapefile = columna[4:] + "_" + nombre_dominio

        # Crear un GeoDataFrame con las coordenadas X e Y
        geometria = [Point(xy) for xy in zip(df[columna_x], df[columna_y])]
        gdf = gpd.GeoDataFrame(df, geometry=geometria, crs=crs)  # WGS84

        # Exportar el Shapefile
        ruta_shapefile = f"C:/10mo/1_TESIS 2/1_GEOQUIMICA/2_NOPARAMETRICO/SHP/{nombre_shapefile}.shp"  # Reemplaza con la ruta de salida
        gdf.to_file(ruta_shapefile)

        print(f"Shapefile '{nombre_shapefile}' exportado exitosamente en {ruta_shapefile}.")
